# Q1: EDA và Xử lý Dữ liệu - Dự đoán Giá Nhà

## Mục tiêu
- Phân tích khám phá dữ liệu (EDA)
- Xử lý dữ liệu thiếu và ngoại lai
- Chuẩn bị dữ liệu cho mô hình prediction intervals

In [ ]:
# Import thư viện cần thiết
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# Cấu hình hiển thị
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [ ]:
# Tải dữ liệu từ Kaggle
# Lưu ý: Bạn cần download dữ liệu từ cuộc thi Kaggle trước
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

print(f"Kích thước tập train: {train_df.shape}")
print(f"Kích thước tập test: {test_df.shape}")
print(f"\nCột trong train: {list(train_df.columns)}")

## 1. Phân tích Thống kê Mô tả

In [ ]:
# Thông tin cơ bản về dataset
def basic_info_analysis(df, name="Dataset"):
    print(f"=== THÔNG TIN CƠ BẢN - {name} ===")
    print(f"Kích thước: {df.shape}")
    print(f"Kiểu dữ liệu:")
    print(df.dtypes.value_counts())
    print(f"\nDữ liệu thiếu:")
    missing = df.isnull().sum()
    missing_percent = (missing / len(df)) * 100
    missing_df = pd.DataFrame({
        'Missing_Count': missing,
        'Missing_Percent': missing_percent
    })
    print(missing_df[missing_df['Missing_Count'] > 0].sort_values('Missing_Count', ascending=False))
    return missing_df

# Phân tích train set
train_missing = basic_info_analysis(train_df, "TRAIN SET")
print("\n" + "="*50 + "\n")
test_missing = basic_info_analysis(test_df, "TEST SET")

In [ ]:
# Thống kê mô tả cho biến mục tiêu (SalePrice)
if 'SalePrice' in train_df.columns:
    print("=== THỐNG KÊ SALEPRICE ===")
    print(train_df['SalePrice'].describe())
    print(f"\nSkewness: {train_df['SalePrice'].skew():.3f}")
    print(f"Kurtosis: {train_df['SalePrice'].kurtosis():.3f}")
    
    # Phân tích outliers
    Q1 = train_df['SalePrice'].quantile(0.25)
    Q3 = train_df['SalePrice'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = train_df[(train_df['SalePrice'] < lower_bound) | (train_df['SalePrice'] > upper_bound)]
    print(f"\nSố lượng outliers: {len(outliers)} ({len(outliers)/len(train_df)*100:.2f}%)")

## 2. Phân tích Phân phối Giá Nhà

In [ ]:
# Phân tích phân phối SalePrice
def analyze_price_distribution(df):
    if 'SalePrice' not in df.columns:
        print("Không có cột SalePrice trong dataset")
        return
    
    fig, axes = plt.subplots(2, 3, figsize=(20, 12))
    
    # 1. Histogram gốc
    axes[0,0].hist(df['SalePrice'], bins=50, alpha=0.7, color='skyblue', edgecolor='black')
    axes[0,0].set_title('Phân phối Giá Nhà (Gốc)', fontsize=14, fontweight='bold')
    axes[0,0].set_xlabel('Giá nhà ($)')
    axes[0,0].set_ylabel('Tần suất')
    axes[0,0].grid(True, alpha=0.3)
    
    # 2. Log transformation
    log_price = np.log(df['SalePrice'])
    axes[0,1].hist(log_price, bins=50, alpha=0.7, color='lightcoral', edgecolor='black')
    axes[0,1].set_title('Phân phối Log(Giá)', fontsize=14, fontweight='bold')
    axes[0,1].set_xlabel('Log(Giá nhà)')
    axes[0,1].set_ylabel('Tần suất')
    axes[0,1].grid(True, alpha=0.3)
    
    # 3. Box plot
    axes[0,2].boxplot(df['SalePrice'])
    axes[0,2].set_title('Box Plot Giá Nhà', fontsize=14, fontweight='bold')
    axes[0,2].set_ylabel('Giá nhà ($)')
    axes[0,2].grid(True, alpha=0.3)
    
    # 4. Q-Q plot gốc
    stats.probplot(df['SalePrice'], dist="norm", plot=axes[1,0])
    axes[1,0].set_title('Q-Q Plot (Gốc)', fontsize=14, fontweight='bold')
    axes[1,0].grid(True, alpha=0.3)
    
    # 5. Q-Q plot log
    stats.probplot(log_price, dist="norm", plot=axes[1,1])
    axes[1,1].set_title('Q-Q Plot (Log)', fontsize=14, fontweight='bold')
    axes[1,1].grid(True, alpha=0.3)
    
    # 6. Density plot
    axes[1,2].plot(df['SalePrice'], stats.gaussian_kde(df['SalePrice'])(df['SalePrice']), 'b-', alpha=0.7)
    axes[1,2].set_title('Density Plot', fontsize=14, fontweight='bold')
    axes[1,2].set_xlabel('Giá nhà ($)')
    axes[1,2].set_ylabel('Density')
    axes[1,2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Thống kê so sánh
    print("=== SO SÁNH PHÂN PHỐI ===")
    print(f"Skewness gốc: {df['SalePrice'].skew():.3f}")
    print(f"Skewness log: {log_price.skew():.3f}")
    print(f"Kurtosis gốc: {df['SalePrice'].kurtosis():.3f}")
    print(f"Kurtosis log: {log_price.kurtosis():.3f}")

analyze_price_distribution(train_df)

## 3. Phân tích Tương quan

In [ ]:
# Phân tích tương quan với SalePrice
def correlation_analysis(df):
    if 'SalePrice' not in df.columns:
        print("Không có cột SalePrice để phân tích tương quan")
        return
    
    # Chỉ lấy các cột số
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    corr_matrix = df[numeric_cols].corr()
    
    # Vẽ heatmap
    plt.figure(figsize=(16, 12))
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
    sns.heatmap(corr_matrix, mask=mask, annot=True, cmap='coolwarm', center=0,
                square=True, linewidths=0.5, cbar_kws={"shrink": .8}, fmt='.2f')
    plt.title('Ma trận Tương quan', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Top correlations với SalePrice
    price_corr = corr_matrix['SalePrice'].abs().sort_values(ascending=False)
    print("=== TOP 15 FEATURES TƯƠNG QUAN CAO NHẤT VỚI SALEPRICE ===")
    for i, (feature, corr) in enumerate(price_corr.head(16).items()):
        if feature != 'SalePrice':
            print(f"{i:2d}. {feature:20s}: {corr:.4f}")
    
    return price_corr.head(16)

top_corr = correlation_analysis(train_df)

In [ ]:
# Vẽ scatter plots cho top features
def plot_top_correlations(df, top_features, n_plots=6):
    if 'SalePrice' not in df.columns:
        return
    
    # Lấy top features (loại trừ SalePrice)
    features_to_plot = [f for f in top_features.index[:n_plots+1] if f != 'SalePrice'][:n_plots]
    
    fig, axes = plt.subplots(2, 3, figsize=(20, 12))
    axes = axes.flatten()
    
    for i, feature in enumerate(features_to_plot):
        if feature in df.columns:
            axes[i].scatter(df[feature], df['SalePrice'], alpha=0.6, s=30)
            axes[i].set_xlabel(feature)
            axes[i].set_ylabel('SalePrice')
            axes[i].set_title(f'{feature} vs SalePrice\n(Corr: {top_features[feature]:.3f})')
            axes[i].grid(True, alpha=0.3)
            
            # Thêm trend line
            z = np.polyfit(df[feature].fillna(df[feature].median()), df['SalePrice'], 1)
            p = np.poly1d(z)
            axes[i].plot(df[feature], p(df[feature]), "r--", alpha=0.8)
    
    plt.tight_layout()
    plt.show()

plot_top_correlations(train_df, top_corr)

## 4. Xử lý Dữ liệu Thiếu

In [ ]:
# Xử lý dữ liệu thiếu
def handle_missing_values(train_df, test_df):
    # Kết hợp train và test để xử lý đồng nhất
    all_data = pd.concat([train_df, test_df], sort=False, ignore_index=True)
    
    # Phân loại các cột theo kiểu dữ liệu
    numeric_cols = all_data.select_dtypes(include=[np.number]).columns.tolist()
    categorical_cols = all_data.select_dtypes(include=['object']).columns.tolist()
    
    print(f"Số cột số: {len(numeric_cols)}")
    print(f"Số cột categorical: {len(categorical_cols)}")
    
    # Xử lý missing values cho numeric columns
    for col in numeric_cols:
        if col != 'SalePrice':  # Không xử lý target variable
            missing_count = all_data[col].isnull().sum()
            if missing_count > 0:
                # Dùng median để fill missing values
                median_val = all_data[col].median()
                all_data[col].fillna(median_val, inplace=True)
                print(f"Filled {missing_count} missing values in {col} with median: {median_val}")
    
    # Xử lý missing values cho categorical columns
    for col in categorical_cols:
        missing_count = all_data[col].isnull().sum()
        if missing_count > 0:
            # Dùng mode để fill missing values
            mode_val = all_data[col].mode().iloc[0] if len(all_data[col].mode()) > 0 else 'Unknown'
            all_data[col].fillna(mode_val, inplace=True)
            print(f"Filled {missing_count} missing values in {col} with mode: {mode_val}")
    
    # Tách lại train và test
    train_processed = all_data[:len(train_df)].copy()
    test_processed = all_data[len(train_df):].copy()
    
    return train_processed, test_processed

train_processed, test_processed = handle_missing_values(train_df, test_df)
print(f"\nSau xử lý - Train missing values: {train_processed.isnull().sum().sum()}")
print(f"Sau xử lý - Test missing values: {test_processed.isnull().sum().sum()}")

## 5. Feature Engineering

In [ ]:
# Feature Engineering
def feature_engineering(train_df, test_df):
    train_fe = train_df.copy()
    test_fe = test_df.copy()
    
    # Tạo features mới cho cả train và test
    for df in [train_fe, test_fe]:
        # 1. Total area features
        if all(col in df.columns for col in ['1stFlrSF', '2ndFlrSF']):
            df['TotalSF'] = df['1stFlrSF'] + df['2ndFlrSF']
        
        if all(col in df.columns for col in ['TotalBsmtSF', 'GrLivArea']):
            df['TotalArea'] = df['TotalBsmtSF'] + df['GrLivArea']
        
        # 2. Age features
        if 'YearBuilt' in df.columns:
            df['HouseAge'] = 2024 - df['YearBuilt']
        
        if 'YearRemodAdd' in df.columns:
            df['RemodAge'] = 2024 - df['YearRemodAdd']
        
        # 3. Bathroom features
        if all(col in df.columns for col in ['FullBath', 'HalfBath']):
            df['TotalBath'] = df['FullBath'] + 0.5 * df['HalfBath']
        
        # 4. Quality features
        if all(col in df.columns for col in ['OverallQual', 'OverallCond']):
            df['QualCondProduct'] = df['OverallQual'] * df['OverallCond']
        
        # 5. Garage features
        if 'GarageArea' in df.columns and 'GarageCars' in df.columns:
            df['GarageAreaPerCar'] = df['GarageArea'] / (df['GarageCars'] + 1)  # +1 để tránh chia 0
    
    print("=== FEATURES MỚI ĐÃ TẠO ===")
    new_features = ['TotalSF', 'TotalArea', 'HouseAge', 'RemodAge', 'TotalBath', 'QualCondProduct', 'GarageAreaPerCar']
    for feature in new_features:
        if feature in train_fe.columns:
            print(f"✓ {feature}")
    
    return train_fe, test_fe

train_fe, test_fe = feature_engineering(train_processed, test_processed)

## 6. Xử lý Outliers

In [ ]:
# Phát hiện và xử lý outliers
def detect_outliers(df, columns, method='IQR'):
    outliers_dict = {}
    
    for col in columns:
        if col in df.columns and df[col].dtype in ['int64', 'float64']:
            if method == 'IQR':
                Q1 = df[col].quantile(0.25)
                Q3 = df[col].quantile(0.75)
                IQR = Q3 - Q1
                lower_bound = Q1 - 1.5 * IQR
                upper_bound = Q3 + 1.5 * IQR
                outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)].index
            elif method == 'Z-score':
                z_scores = np.abs(stats.zscore(df[col]))
                outliers = df[z_scores > 3].index
            
            outliers_dict[col] = outliers
    
    return outliers_dict

# Phát hiện outliers trong SalePrice và top features
if 'SalePrice' in train_fe.columns:
    important_cols = ['SalePrice'] + [col for col in top_corr.index[:6] if col != 'SalePrice' and col in train_fe.columns]
    outliers_dict = detect_outliers(train_fe, important_cols)
    
    print("=== OUTLIERS PHÁT HIỆN ===")
    for col, outliers in outliers_dict.items():
        print(f"{col}: {len(outliers)} outliers ({len(outliers)/len(train_fe)*100:.2f}%)")
    
    # Visualize outliers
    fig, axes = plt.subplots(2, 3, figsize=(20, 12))
    axes = axes.flatten()
    
    for i, col in enumerate(important_cols[:6]):
        if col in train_fe.columns:
            axes[i].boxplot(train_fe[col].dropna())
            axes[i].set_title(f'Box Plot - {col}')
            axes[i].set_ylabel(col)
            axes[i].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 7. Encoding Categorical Variables

In [ ]:
# Encode categorical variables
def encode_categorical_features(train_df, test_df):
    train_encoded = train_df.copy()
    test_encoded = test_df.copy()
    
    # Lấy các cột categorical
    categorical_cols = train_encoded.select_dtypes(include=['object']).columns.tolist()
    
    print(f"=== ENCODING {len(categorical_cols)} CATEGORICAL FEATURES ===")
    
    # Kết hợp train và test để đảm bảo consistent encoding
    all_data = pd.concat([train_encoded, test_encoded], sort=False)
    
    for col in categorical_cols:
        # Kiểm tra số lượng unique values
        unique_count = all_data[col].nunique()
        print(f"{col}: {unique_count} unique values")
        
        if unique_count <= 10:  # One-hot encoding cho ít categories
            dummies = pd.get_dummies(all_data[col], prefix=col, dummy_na=True)
            all_data = pd.concat([all_data.drop(col, axis=1), dummies], axis=1)
        else:  # Label encoding cho nhiều categories
            le = LabelEncoder()
            all_data[col] = le.fit_transform(all_data[col].astype(str))
    
    # Tách lại train và test
    train_encoded = all_data[:len(train_df)].copy()
    test_encoded = all_data[len(train_df):].copy()
    
    print(f"\nSau encoding:")
    print(f"Train shape: {train_encoded.shape}")
    print(f"Test shape: {test_encoded.shape}")
    
    return train_encoded, test_encoded

train_encoded, test_encoded = encode_categorical_features(train_fe, test_fe)

## 8. Scaling Features

In [ ]:
# Feature scaling
def scale_features(train_df, test_df, target_col='SalePrice', method='robust'):
    # Tách features và target
    if target_col in train_df.columns:
        X_train = train_df.drop(target_col, axis=1)
        y_train = train_df[target_col]
    else:
        X_train = train_df.copy()
        y_train = None
    X_test = test_df.copy()
    
    # Chỉ scale numeric columns
    numeric_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
    
    if method == 'robust':
        scaler = RobustScaler()
    else:
        scaler = StandardScaler()
    
    # Fit trên train, transform cả train và test
    X_train_scaled = X_train.copy()
    X_test_scaled = X_test.copy()
    
    X_train_scaled[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
    X_test_scaled[numeric_cols] = scaler.transform(X_test[numeric_cols])
    
    print(f"=== SCALING COMPLETED USING {method.upper()} SCALER ===")
    print(f"Scaled {len(numeric_cols)} numeric features")
    
    return X_train_scaled, X_test_scaled, y_train, scaler

X_train_scaled, X_test_scaled, y_train, scaler = scale_features(train_encoded, test_encoded)

## 9. Lưu Dữ liệu Đã Xử lý

In [ ]:
# Lưu dữ liệu đã xử lý
def save_processed_data(X_train, X_test, y_train, scaler):
    # Tạo thư mục nếu chưa có
    import os
    os.makedirs('processed_data', exist_ok=True)
    
    # Lưu features
    X_train.to_csv('processed_data/X_train_processed.csv', index=False)
    X_test.to_csv('processed_data/X_test_processed.csv', index=False)
    
    # Lưu target
    if y_train is not None:
        pd.DataFrame({'SalePrice': y_train}).to_csv('processed_data/y_train.csv', index=False)
    
    # Lưu scaler
    import joblib
    joblib.dump(scaler, 'processed_data/scaler.pkl')
    
    print("=== DỮ LIỆU ĐÃ ĐƯỢC LUU ===")
    print(f"X_train shape: {X_train.shape}")
    print(f"X_test shape: {X_test.shape}")
    if y_train is not None:
        print(f"y_train shape: {y_train.shape}")
    
    return True

save_processed_data(X_train_scaled, X_test_scaled, y_train, scaler)

## 10. Tóm tắt EDA

In [ ]:
# Tóm tắt kết quả EDA
def eda_summary():
    print("=" * 60)
    print("                    TÓM TẮT EDA")
    print("=" * 60)
    
    print("\n📊 THỐNG KÊ DATASET:")
    print(f"   • Train samples: {len(train_df)}")
    print(f"   • Test samples: {len(test_df)}")
    print(f"   • Features sau xử lý: {X_train_scaled.shape[1]}")
    
    if 'SalePrice' in train_df.columns:
        print(f"\n💰 PHÂN TÍCH GIÁ NHÀ:")
        print(f"   • Giá trung bình: ${train_df['SalePrice'].mean():,.0f}")
        print(f"   • Giá median: ${train_df['SalePrice'].median():,.0f}")
        print(f"   • Khoảng giá: ${train_df['SalePrice'].min():,.0f} - ${train_df['SalePrice'].max():,.0f}")
        print(f"   • Skewness: {train_df['SalePrice'].skew():.3f}")
    
    print(f"\n🔧 XỬ LÝ DỮ LIỆU:")
    print(f"   • Missing values: Đã xử lý")
    print(f"   • Categorical encoding: Đã thực hiện")
    print(f"   • Feature scaling: RobustScaler")
    print(f"   • Feature engineering: Đã tạo features mới")
    
    if 'top_corr' in globals():
        print(f"\n📈 TOP 5 FEATURES QUAN TRỌNG:")
        for i, (feature, corr) in enumerate(list(top_corr.items())[1:6]):
            print(f"   {i+1}. {feature}: {corr:.3f}")
    
    print(f"\n✅ CHUẨN BỊ CHO MODELING:")
    print(f"   • Dữ liệu đã được lưu vào thư mục 'processed_data'")
    print(f"   • Sẵn sàng cho bước huấn luyện mô hình")
    
    print("\n" + "=" * 60)

eda_summary()